# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
# Reference the file where the CSV is located
city_csv_path = "../WeatherPy/output_data/City_Data.csv"

# Import the data into a Pandas DataFrame
##city_data_df = pd.read_csv(city_csv_path, encoding="utf-8")
city_data_df = pd.read_csv(city_csv_path, encoding="utf-8")

# Show the data
city_data_df

,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kapaa,22.08,-159.32,84.20,66,40,12.75,US,1596406323
1,Upernavik,72.79,-56.15,44.08,69,93,6.62,GL,1596406323
2,Albany,42.60,-73.97,87.01,65,47,9.80,US,1596406324
3,Punta Arenas,-53.15,-70.92,39.20,80,24,6.93,CL,1596406043
4,Naze,28.37,129.48,82.40,78,40,9.17,JP,1596406216
...,...,...,...,...,...,...,...,...,...
547,Kahului,20.89,-156.47,87.80,52,40,23.04,US,1596406419
548,Hualmay,-11.10,-77.61,64.08,73,18,10.20,PE,1596406465
549,Punta de Piedra,7.62,-71.49,80.83,88,59,2.55,VE,1596406466
550,Iwaki,37.05,140.88,77.00,88,20,3.36,JP,1596406466


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"].astype(float)

In [10]:
# Create a humidity Heatmap layer
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=humidity.max(),point_radius=2))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# Create a new dataframe populated by rows fitting my ideal weather conditions
ideal_weather_df = pd.DataFrame(city_data_df.loc[(city_data_df["Humidity"] < 45) & (city_data_df["Cloudiness"] < 20) & \
                                                 ((city_data_df["Temp"] < 85) & (city_data_df["Temp"] >= 70)) & \
                                                (city_data_df["Wind Speed"] <= 10)])
ideal_weather_df = ideal_weather_df.dropna().reset_index(drop = True)
ideal_weather_df

,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Gazli,40.13,63.45,78.37,24,0,7.96,UZ,1596406335
1,Vallenar,-28.57,-70.76,73.49,29,5,3.22,CL,1596406363
2,Granada,37.19,-3.61,84.20,14,0,8.05,ES,1596406365
3,Shetpe,44.17,52.12,78.03,39,0,6.38,KZ,1596406381
4,Conceição do Araguaia,-8.26,-49.26,83.30,38,0,3.98,BR,1596406409
5,Gazojak,41.19,61.40,80.55,23,0,6.51,TM,1596406414
6,Birjand,32.87,59.22,71.60,21,0,4.70,IR,1596406418
7,Horqueta,-23.34,-57.06,77.74,40,0,5.08,PY,1596406420
8,Presidente Médici,-11.18,-61.90,84.33,26,0,3.51,BR,1596406428
9,Jacareacanga,-6.22,-57.75,81.25,38,5,0.69,BR,1596406429


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
# Add column to the dataframe to hold the hotel name
ideal_weather_df["Hotel Name"] = ""

# Create a dataframe called hotel_df
hotel_df = pd.DataFrame(ideal_weather_df)
hotel_df


,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Gazli,40.13,63.45,78.37,24,0,7.96,UZ,1596406335,
1,Vallenar,-28.57,-70.76,73.49,29,5,3.22,CL,1596406363,
2,Granada,37.19,-3.61,84.20,14,0,8.05,ES,1596406365,
3,Shetpe,44.17,52.12,78.03,39,0,6.38,KZ,1596406381,
4,Conceição do Araguaia,-8.26,-49.26,83.30,38,0,3.98,BR,1596406409,
5,Gazojak,41.19,61.40,80.55,23,0,6.51,TM,1596406414,
6,Birjand,32.87,59.22,71.60,21,0,4.70,IR,1596406418,
7,Horqueta,-23.34,-57.06,77.74,40,0,5.08,PY,1596406420,
8,Presidente Médici,-11.18,-61.90,84.33,26,0,3.51,BR,1596406428,
9,Jacareacanga,-6.22,-57.75,81.25,38,5,0.69,BR,1596406429,


In [13]:
# Add column to the dataframe that has the combined latitude and longitude in the format the Google Places API requires
combined_lat_lng = hotel_df["Lat"].astype(str) + "," + hotel_df["Lng"].astype(str)
hotel_df["Lat Lng"] = combined_lat_lng

# Create the base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set up a dictionary to hold the parameters to be passed in the URL
params = {
    "keyword": "Hotel",
    "rankby": "distance",
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through the dataframe rows
for index, row in hotel_df.iterrows():

    # get lat, lng from df
    target_coordinates = row["Lat Lng"]

    # add location to params dict
    params['location'] = target_coordinates

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        pass

# Show the results in the dataframe to make sure hotels were added
hotel_df

,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Lat Lng
0,Gazli,40.13,63.45,78.37,24,0,7.96,UZ,1596406335,,"40.13,63.45"
1,Vallenar,-28.57,-70.76,73.49,29,5,3.22,CL,1596406363,Hotel Del Marques,"-28.57,-70.76"
2,Granada,37.19,-3.61,84.20,14,0,8.05,ES,1596406365,Allegro Granada,"37.19,-3.61"
3,Shetpe,44.17,52.12,78.03,39,0,6.38,KZ,1596406381,Gostinitsa Nurlan,"44.17,52.12"
4,Conceição do Araguaia,-8.26,-49.26,83.30,38,0,3.98,BR,1596406409,Hotel Pousada Do Sol,"-8.26,-49.26"
5,Gazojak,41.19,61.40,80.55,23,0,6.51,TM,1596406414,Khorezm Auto Plaza,"41.19,61.4"
6,Birjand,32.87,59.22,71.60,21,0,4.70,IR,1596406418,Tourism guesthouse Birjand,"32.87,59.22"
7,Horqueta,-23.34,-57.06,77.74,40,0,5.08,PY,1596406420,Fromherz Hotel,"-23.34,-57.06"
8,Presidente Médici,-11.18,-61.90,84.33,26,0,3.51,BR,1596406428,Hotel Dos Viajantes,"-11.18,-61.9"
9,Jacareacanga,-6.22,-57.75,81.25,38,5,0.69,BR,1596406429,Zé Prata,"-6.22,-57.75"


In [14]:
# Replace blank hotel names with NaN, drop those rows in which a hotel could not be found, and then reindex the dataframe
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df = hotel_df.dropna().reset_index(drop = True)

# Show the datafrome
hotel_df

,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Lat Lng
0,Vallenar,-28.57,-70.76,73.49,29,5,3.22,CL,1596406363,Hotel Del Marques,"-28.57,-70.76"
1,Granada,37.19,-3.61,84.20,14,0,8.05,ES,1596406365,Allegro Granada,"37.19,-3.61"
2,Shetpe,44.17,52.12,78.03,39,0,6.38,KZ,1596406381,Gostinitsa Nurlan,"44.17,52.12"
3,Conceição do Araguaia,-8.26,-49.26,83.30,38,0,3.98,BR,1596406409,Hotel Pousada Do Sol,"-8.26,-49.26"
4,Gazojak,41.19,61.40,80.55,23,0,6.51,TM,1596406414,Khorezm Auto Plaza,"41.19,61.4"
5,Birjand,32.87,59.22,71.60,21,0,4.70,IR,1596406418,Tourism guesthouse Birjand,"32.87,59.22"
6,Horqueta,-23.34,-57.06,77.74,40,0,5.08,PY,1596406420,Fromherz Hotel,"-23.34,-57.06"
7,Presidente Médici,-11.18,-61.90,84.33,26,0,3.51,BR,1596406428,Hotel Dos Viajantes,"-11.18,-61.9"
8,Jacareacanga,-6.22,-57.75,81.25,38,5,0.69,BR,1596406429,Zé Prata,"-6.22,-57.75"
9,Itapuranga,-15.56,-49.95,72.70,38,0,5.91,BR,1596406447,Itabras Hotel,"-15.56,-49.95"


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add Humidity map layer from above but set locations in this layer to the city_data_df dataframe latitudes and longitudes
fig.add_layer(gmaps.heatmap_layer(locations=city_data_df[["Lat", "Lng"]], weights=humidity, dissipating=False, max_intensity=humidity.max(),point_radius=2))

# Add marker layer on top of heatmap
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))